In [1]:
import torch
import torch.nn as nn
from Model import model, modelArguments
import time
import re
import math
import triton

In [2]:
fmodel = model(modelArguments)

state_dict = torch.load('./checkpoints/model_weightsBPE0.pth')
new_state_dict = {k.replace('_orig_mod.', ''): v for k, v in state_dict.items()}
fmodel.load_state_dict(new_state_dict, strict=False)

fmodel = fmodel.to('cuda')

/tmp/ipykernel_348/2029478873.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('./checkpoints/model_weightsBPE0.pth')


In [3]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast(tokenizer_file="large_tokenizer.json")


def tokenize(text, second = False):
    ids = tokenizer.encode(text, add_special_tokens=True)
    ids1 = ids
    ids1 = torch.tensor(ids1, dtype=torch.int16)
    
    # # Pad or truncate the sequence to tholtae max_seq_length
    # if ids1.size(0) < modelArguments.max_seq_len:
    #     padding_length = modelArguments.max_seq_len - ids1.size(0)
    #     ids1 = torch.cat([ids1, torch.zeros(padding_length, dtype=torch.int16)])
        
    # elif ids1.size(0) >= modelArguments.max_seq_len:
    #     ids1 = ids1[:modelArguments.max_seq_len]
    #     ids1[-1] = 2
    return ids1

/home/frosthead/.conda/envs/DL/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def id_to_token(x):
    s = ''
    for i in x[0]:
        # j = torch.argmax(i)
        s += tokenizer.decode(i)

    return s

def id_to_token_M(x):
    s = ''
    for i in x[0]:
        j = torch.argmax(i)
        s += tokenizer.decode(j)

    return s

In [5]:
fmodel.eval()

model(
  (shared_emb): shared_emb()
  (enc_layers): ModuleList(
    (0-9): 10 x EncoderBlock(
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (attention): FlashAttention(
        (q_linear): Linear(in_features=512, out_features=512, bias=False)
        (k_linear): Linear(in_features=512, out_features=512, bias=False)
        (v_linear): Linear(in_features=512, out_features=512, bias=False)
        (out_linear): Linear(in_features=512, out_features=512, bias=False)
      )
      (feedforward): FeedForward(
        (fc1): Linear(in_features=512, out_features=2048, bias=False)
        (fc2): Linear(in_features=2048, out_features=512, bias=False)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
)

In [13]:
# test sentence

sentence = """complete this story \n Once there lived a young boy named ayush"""
generated_id  = tokenize(sentence).to(torch.long).to('cuda').unsqueeze(0)
generated_ids = torch.cat([generated_id for i in range(4)], dim=0)

print(generated_ids.shape)
# print(generated_ids)


t0 = time.time()

for _ in range(192): #modelArguments.max_seq_len-generated_ids.size(1)):  # Generate up to 20 tokens
    with torch.no_grad():
        logits = fmodel(generated_ids)  # Pass the current sequence to the model
        next_token_logits = logits[:, -1, :]  # Get logits for the last token


    # Predict the next token (greedy decoding)
    next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(1)  # Shape: [1, 1]

    # Append the predicted token to the sequence
    generated_ids = torch.cat([generated_ids, next_token_id], dim=-1)

    # Stop if EOS token is predicted
    # if next_token_id.item() == 2:
    #     print("eos")
    #     break

t1 = time.time()
# Decode the generated sequence
output_text = tokenizer.decode(generated_ids[0])
print("Generated text:", output_text)
t = (t1-t0)*1000
print("time taken ", t)


torch.Size([4, 13])
Generated text: complete this story 
 Once there lived a young boy named ayush of his house. The boy was a young boy who had a small house. The boy was a young boy. The boy was a very young, but he was a young boy. He was a young boy who had a house. He was a young boy. He was a young boy who had a house. He was a young boy. He was a young boy who had a house. He was a young boy, but he was a young boy. He was a young boy. He was a young boy. He was a young boy. He was a young boy, and he was a young boy. He was a young boy. He was a young boy, and he had a little boy. He was a young boy. He was a young boy. He was a young boy. He was a young boy. He was a young boy. He was a young boy. He was a young boy. He was a young boy. He was
time taken  5871.848821640015


In [14]:
generated_ids.shape

torch.Size([4, 205])

In [15]:
next_token_id.shape

torch.Size([4, 1])

In [16]:
output_text = tokenizer.decode(generated_ids[0])
print("Generated text:", output_text)

Generated text: complete this story 
 Once there lived a young boy named ayush of his house. The boy was a young boy who had a small house. The boy was a young boy. The boy was a very young, but he was a young boy. He was a young boy who had a house. He was a young boy. He was a young boy who had a house. He was a young boy. He was a young boy who had a house. He was a young boy, but he was a young boy. He was a young boy. He was a young boy. He was a young boy. He was a young boy, and he was a young boy. He was a young boy. He was a young boy, and he had a little boy. He was a young boy. He was a young boy. He was a young boy. He was a young boy. He was a young boy. He was a young boy. He was a young boy. He was a young boy. He was


In [17]:
output_text = tokenizer.decode(generated_ids[1])
print("Generated text:", output_text)

Generated text: complete this story 
 Once there lived a young boy named ayush her. 
 

I had a little to the house. 
I had a little to the house. 
I had a little to see. 
I had a little to the house. 
I had a little to the house. 
I had a little to see. 
I had a little to the house. 
I had a little to see. 
I had a little to the house. 
I had a little to the house. 
I had a little to the house. 
I had a little to the house. 
I had a little to it. 
I had a little to the house. 
I had a little to the house. 
I had a little to the house. 
I had a little to the house. 
I had a little to the house. 
I had a little to the house. 
I


In [18]:
output_text = tokenizer.decode(generated_ids[2])
print("Generated text:", output_text)

Generated text: complete this story 
 Once there lived a young boy named ayush of a boy. He was a young boy, he was a young boy who had a little boy who had a lot of his life. He was a little boy, but he had a little boy. He had a little boy. He was a little boy. He had a little boy. He had a little boy. He had a little boy. He had a little boy. He had a little boy. He had a little boy. He had a little boy. He was a little boy. He had a little boy. He had a little boy. He was a little boy. He was a little boy. He was a young boy. He was a little boy. He had a little boy. He was a young boy. He was a little boy. He had a little boy. He was a little boy. He had a little boy. He was a little boy. He had a little boy. He was a little boy. He


In [19]:
output_text = tokenizer.decode(generated_ids[3])
print("Generated text:", output_text)

Generated text: complete this story 
 Once there lived a young boy named ayush the old girl, the boy's friend was a young boy who was a young boy who was a young boy. The boy was a young boy who was a young boy. The boy was a young boy who was a young boy who was a young boy who was a young boy. He was a young boy. He had a new boy's friends, but he had a little to her. He was a young boy. He had a new boy's friend, but he had a little to see his friends. He was a young boy. He had a little of his friends. He had a new boy who was a young boy. He was a young boy who was a young boy. He had a new boy. He had a little to see his friends. He had a new boy's name. He was a young boy. He had a new boy. He had a new boy. He was a young boy. He had a little boy.
